In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = (pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
         .sort_values(['time_id', 'investment_id'])
         .drop(columns=['row_id'])
         .query('time_id > 599')
         .reset_index(drop=True));


In [ ]:
train_df.drop(['time_id'],axis=1,inplace=True)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components='mle')
principalComponents = pca.fit_transform(train_df.loc[:,train_df.columns!='target'])

In [ ]:
principalDf = pd.DataFrame(data = principalComponents)

In [ ]:
from sklearn.preprocessing import StandardScaler
# define data
scaler = StandardScaler()
# transform data
principalDf = scaler.fit_transform(principalDf)
principalDf = pd.DataFrame(data = principalDf)

In [ ]:
y=train_df['target']
del(train_df)

In [ ]:
import xgboost
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(principalDf,y)


In [ ]:

#principalComponentstest = pca.fit_transform(test_df)
#principalDfTest = pd.DataFrame(data = principalComponentstest)
#principalDfTest = scaler.fit_transform(principalDfTest)
#prediction=model.predict(principalDfTest)


In [ ]:
#prediction

In [ ]:
#model.save_model('./model_pca')

In [ ]:
def reduce_df(df):
    principalComponentstest = pca.transform(df)
    principalDfTest = pd.DataFrame(data = principalComponentstest)
    principalDfTest = scaler.transform(principalDfTest)
    principalDfTest=pd.DataFrame(data = principalDfTest)
    return principalDfTest


In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment  
iter_test = env.iter_test()  

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df_reduced=reduce_df(test_df.loc[:,test_df.columns!='row_id'])
    sample_prediction_df['target'] = model.predict(test_df_reduced)  
    env.predict(sample_prediction_df)   

In [ ]:
submission=pd.read_csv("./submission.csv")

In [ ]:
submission